<a href="https://colab.research.google.com/github/Chromis07/dataStudy/blob/main/python/hands_on_data/%ED%83%80%EC%8A%88(%EA%B3%B5%EC%9C%A0_%EC%9E%90%EC%A0%84%EA%B1%B0)_%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 라이브러리 import
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

In [14]:
# 크롤링 정보를 저장할 데이터프레임 선언
data = pd.DataFrame(columns=['스테이션명', '위치',	'상태정보', '위도', '경도'])

# 크롤링 할 타슈 사이트 주소
url_tashu = "http://www.tashu.or.kr/userStationAction.do?process=list" # http(s) 꼭 제거!!!

# 페이지 이동 시 추가로 적어줘야 하는 url
url_add = '&page='

# 전체 페이지 수
total_page = 27

# 27 page에 대한 정보를 추출하는 for문
for index in range(1, total_page+1) :
  if index == 1 :
    url = url_tashu # 첫번재 페이지
  else : 
    url = url_tashu + url_add + str(index)

  # url에서 hrml 추출
  html = urlopen(url)

  # html을 파싱하여 데이터로 변환
  bsObject = BeautifulSoup(html, 'html.parser', from_encoding='utf-8')

  # history_table이라고 하는 class를 가진 table만 찾기 (div 클래스로 가야하지만 아래 에도 class명 존재하니까 가능)
  table = bsObject.find_all('table', {'class', 'history_table'})

  # table 내에서 tr 추출
  tr = table[0].find_all('tr') # table_history에서 한 개만 있었으니 [0] 가능
  
  # 첫 tr은 contents가 아니라, 데이터 정보를 의미하기 때문에 제외 [1:tr 수까지]
  tr = tr[1:len(tr)]

  # 한 페이지의 정보를 크롤링
  for index_tr in range(0, len(tr)) :
    td = tr[index_tr].find_all('td')

    # 장소
    station = td[0].text.split('.')[1] # '1.무역전시관입구(택시승강장)'

    # 위치
    location = td[1].text

    # 상태
    condition = td[2].text

    # 좌표 추출
    geo = td[3].a.attrs['href'].split('(')[1] # '36.374389', '127.392375'); 추출 
    
    # 위도 ,경도 추출을 위한 데이터 클렌징
    geo = re.sub('[;)]', '', geo) # '36.374389', '127.392375'
    detail_location = geo.replace("'", "") # 작은 따옴표 삭제

    # 위도 Latitude
    lat = detail_location.split(',')[0]

    # 경도 Longtitude
    lon = detail_location.split(',')[1]

    # data에 crawling 결과 저장
    data = data.append({'스테이션명' : station,
                        '위치' : location,
                        '상태정보' : condition,
                        '위도' : lat,
                        '경도' : lon},
                       ignore_index=True)
    print('Complete of ' + str(index_tr+1) + ' information with ' +
          str(index) + 'Page')
    
  # 크롤링 막히는 것 방지
  time.sleep(1.0)

print('-'*60)
print(data)


Complete of 1 information with 1Page
Complete of 2 information with 1Page
Complete of 3 information with 1Page
Complete of 4 information with 1Page
Complete of 5 information with 1Page
Complete of 6 information with 1Page
Complete of 7 information with 1Page
Complete of 8 information with 1Page
Complete of 9 information with 1Page
Complete of 10 information with 1Page
Complete of 1 information with 2Page
Complete of 2 information with 2Page
Complete of 3 information with 2Page
Complete of 4 information with 2Page
Complete of 5 information with 2Page
Complete of 6 information with 2Page
Complete of 7 information with 2Page
Complete of 8 information with 2Page
Complete of 9 information with 2Page
Complete of 10 information with 2Page
Complete of 1 information with 3Page
Complete of 2 information with 3Page
Complete of 3 information with 3Page
Complete of 4 information with 3Page
Complete of 5 information with 3Page
Complete of 6 information with 3Page
Complete of 7 information with 3Page

크롤링 데이터 저장하기

In [15]:
data.to_csv('TASHU.csv', encoding = 'utf-8-sig')